# 🔧 04: Tool Calling Basics

Learn how to extend your LLM's capabilities by giving it access to built-in tools for calculations, text transformations, and more.

In [12]:
# Auto-reload modules when they change (helpful during SDK development)
%load_ext autoreload
%autoreload 2

print("✅ Auto-reload enabled - SDK changes will be picked up automatically!")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
✅ Auto-reload enabled - SDK changes will be picked up automatically!


## 📋 Learning Objectives

By the end of this notebook, you will be able to:

- [ ] Understand what tools are and why they're useful
- [ ] Use the built-in `math_calculator` tool for arithmetic
- [ ] Use the `text_transformer` tool for text manipulation
- [ ] Use the `char_counter` tool for text analysis
- [ ] See how tools are automatically executed by the SDK
- [ ] Inspect tool calls in full responses
- [ ] Combine multiple tools in a single conversation

## 🎯 Prerequisites

- Completed notebooks 02 (Basic Chat) and 03 (Conversation History)
- Understanding of chat and conversation history
- LM Studio running with a model that supports function calling

## ⏱️ Estimated Time: 15 minutes

## 1️⃣ What Are Tools?

**Tools** (also called function calling) let LLMs interact with external functions. This solves key limitations:

| Without Tools | With Tools |
|---------------|------------|
| ❌ Bad at precise math | ✅ Use calculator for exact answers |
| ❌ Can't access real-time data | ✅ Call APIs for current information |
| ❌ Can't modify files | ✅ Use file operations |
| ❌ No access to external systems | ✅ Integrate with databases, services |

**How it works:**
1. You give the LLM a list of available tools
2. The LLM decides when to use a tool
3. The LLM generates a tool call with parameters
4. The SDK executes the tool automatically
5. The result goes back to the LLM
6. The LLM incorporates it into the response

## 2️⃣ Built-in Tool: math_calculator

The `math_calculator` tool evaluates mathematical expressions safely.

In [13]:
from local_llm_sdk import LocalLLMClient

# Create client
client = LocalLLMClient(
    base_url="http://169.254.83.107:1234/v1",
    model="mistralai/magistral-small-2509"
)

# Register built-in tools (pass None to load defaults)
client.register_tools_from(None)

print("✅ Registered tools:")
for tool_name in client.tools.list_tools():
    print(f"  - {tool_name}")

✅ Registered tools:
  - char_counter
  - math_calculator
  - get_weather
  - text_transformer
  - execute_python
  - filesystem_operation


**💡 Quick Tip:** You can also use the convenience function to create a client with tools in one step:

```python
from local_llm_sdk import create_client_with_tools

client = create_client_with_tools(
    base_url="http://169.254.83.107:1234/v1",
    model="mistralai/magistral-small-2509"
)
# Tools are already registered!
```

Now let's use the calculator!

In [14]:
# Ask a math question
response = client.chat("What is 127 multiplied by 893?")
print(response)

# Show tool execution details
client.print_tool_calls()

The result of multiplying 127 by 893 is 113,411.

🔧 Tool Execution Summary (1 call):
  [1] math_calculator(arg1=127, arg2=893, operation=multiply) → result=113411



**🎉 Behind the scenes:**
1. The LLM recognized this needs calculation
2. It called `math_calculator(arg1=127, arg2=893, operation="multiply")`
3. The SDK executed the tool: `127 * 893 = 113411`
4. The LLM received the result
5. The LLM formatted a natural language response

**💡 The `print_tool_calls()` method shows:**
- Which tools were called
- What arguments were passed
- What results were returned
- All in a clean, readable format!

Try `client.print_tool_calls(detailed=True)` for full JSON output.

Let's try more complex calculations:

In [15]:
# Complex expression
response = client.chat("Calculate: (15 + 25) * 3 / 2")
print("Question: Calculate: (15 + 25) * 3 / 2")
print(f"Answer: {response}\n")
client.print_tool_calls()

# With context
response = client.chat(
    "If I have 12 boxes with 24 items each, and I sell them at $3.50 per item, "
    "how much revenue do I make?"
)
print("\nQuestion: Revenue calculation")
print(f"Answer: {response}\n")
client.print_tool_calls()

# Multiple calculations
response = client.chat(
    "What's the area of a rectangle with width 15.5 and height 23.7? "
    "And what's the perimeter?"
)
print("\nQuestion: Rectangle area and perimeter")
print(f"Answer: {response}\n")
client.print_tool_calls()

Question: Calculate: (15 + 25) * 3 / 2
Answer: The result of (15 + 25) * 3 / 2 is 60.0.


🔧 Tool Execution Summary (2 calls):
  [1] math_calculator(arg1=40, arg2=3, operation=multiply) → result=120
  [2] math_calculator(arg1=120, arg2=2, operation=divide) → result=60.0


Question: Revenue calculation
Answer: You have a total of \(12 \times 24 = 288\) items. Selling them at $3.50 per item, your total revenue would be \(\$3.50 \times 288 = \$1008\). So, you make $1008 in revenue.


🔧 Tool Execution Summary (2 calls):
  [1] math_calculator(arg1=12, arg2=24, operation=multiply) → result=288
  [2] math_calculator(arg1=288, arg2=3.5, operation=multiply) → result=1008.0


Question: Rectangle area and perimeter
Answer: The area of the rectangle is \(367.35\) and the perimeter is \(86.4\).


🔧 Tool Execution Summary (5 calls):
  [1] math_calculator(arg1=15.5, arg2=23.7, operation=multiply) → result=367.34999999999997
  [2] math_calculator(arg1=15.5, arg2=23.7, operation=add) → result=39.2
  [3]

## 3️⃣ Built-in Tool: text_transformer

The `text_transformer` tool can uppercase, lowercase, reverse, or count words in text.

In [16]:
# Uppercase transformation
response = client.chat("Convert 'hello world' to uppercase")
print("Uppercase:", response)
client.print_tool_calls()

# Lowercase transformation
response = client.chat("Make 'PYTHON IS AWESOME' all lowercase")
print("\nLowercase:", response)
client.print_tool_calls()

# Reverse text
response = client.chat("Reverse the text: 'artificial intelligence'")
print("\nReverse:", response)
client.print_tool_calls()

# Count words
response = client.chat(
    "How many words are in this sentence: 'The quick brown fox jumps over the lazy dog'"
)
print("\nWord count:", response)
client.print_tool_calls()

Uppercase: The converted text is: HELLO WORLD

🔧 Tool Execution Summary (1 call):
  [1] text_transformer(text=hello world, transform=upper) → original=hello world


Lowercase: The text transformed to lowercase is: python is awesome.

🔧 Tool Execution Summary (1 call):
  [1] text_transformer(text=PYTHON IS AWESOME, transform=lower) → original=PYTHON IS AWESOME


Reverse: The reversed text is: 'ecnegilletnillartifirp'

🔧 Tool Execution Summary (1 call):
  [1] execute_python(code=text = 'artificial intelligence'
reversed_text = text[::-1]
reversed_text) → success=True


Word count: There are 9 words in the sentence: 'The quick brown fox jumps over the lazy dog'.

🔧 Tool Execution Summary (1 call):
  [1] char_counter(text=The quick brown fox jumps over the lazy dog) → text=The quick brown fox jumps over the lazy dog



## 4️⃣ Built-in Tool: char_counter

The `char_counter` tool counts characters in text (with option to include/exclude spaces).

In [17]:
# Count with spaces
response = client.chat("How many characters are in 'Hello, World!' including spaces?")
print("With spaces:", response)
client.print_tool_calls()

# Count without spaces
response = client.chat("How many characters are in 'Hello, World!' excluding spaces?")
print("\nWithout spaces:", response)
client.print_tool_calls()

# Compare lengths
response = client.chat(
    "Which is longer: 'supercalifragilisticexpialidocious' or 'antidisestablishmentarianism'? "
    "Tell me the character count of each."
)
print("\nComparison:", response)
client.print_tool_calls()

With spaces: The phrase 'Hello, World!' has a total of 13 characters, including spaces.

🔧 Tool Execution Summary (1 call):
  [1] char_counter(text=Hello, World!) → text=Hello, World!


Without spaces: There are 12 characters in 'Hello, World!' excluding spaces.

🔧 Tool Execution Summary (1 call):
  [1] char_counter(text=Hello,World!) → text=Hello,World!


Comparison: "supercalifragilisticexpialidocious" is longer with a character count of 34. In comparison, "antidisestablishmentarianism" has a character count of 28.

🔧 Tool Execution Summary (2 calls):
  [1] char_counter(text=supercalifragilisticexpialidocious) → text=supercalifragilisticexpialidocious
  [2] char_counter(text=antidisestablishmentarianism) → text=antidisestablishmentarianism



## 5️⃣ Inspecting Tool Calls

Two ways to see tool execution details:

1. **Quick Summary**: `client.print_tool_calls()` - Clean, readable format
2. **Full Details**: `return_full_response=True` - Complete ChatCompletion object

In [18]:
print("METHOD 1: Quick summary with print_tool_calls()")
print("=" * 70)

response = client.chat("What is 456 * 789 and also uppercase the word 'python'")
print(f"\nResponse: {response}\n")

# Show compact summary
client.print_tool_calls()

# Show detailed version
print("\n\nSame call with detailed=True:")
client.print_tool_calls(detailed=True)

print("\n" + "=" * 70)
print("\nMETHOD 2: Full ChatCompletion object")
print("=" * 70 + "\n")

# Get full response object
response = client.chat(
    "Calculate 15 + 25 and reverse the text 'hello world'",
    return_full_response=True
)

print(f"Model: {response.model}")
print(f"Finish reason: {response.choices[0].finish_reason}")
print(f"Final message: {response.choices[0].message.content}")

# Check tool_calls in the response
message = response.choices[0].message
if message.tool_calls:
    print(f"\n🔧 Tool Calls Made: {len(message.tool_calls)}")
    for i, tool_call in enumerate(message.tool_calls, 1):
        print(f"\nTool Call {i}:")
        print(f"  Function: {tool_call.function.name}")
        print(f"  Arguments: {tool_call.function.arguments}")
        print(f"  ID: {tool_call.id}")

METHOD 1: Quick summary with print_tool_calls()

Response: The result of 456 multiplied by 789 is **359,784**, and the uppercase version of 'python' is **PYTHON**.


🔧 Tool Execution Summary (2 calls):
  [1] math_calculator(arg1=456, arg2=789, operation=multiply) → result=359784
  [2] text_transformer(text=python, transform=upper) → original=python



Same call with detailed=True:

🔧 Tool Execution Summary (2 calls):

[1] math_calculator
    Arguments: {
      "arg1": 456,
      "arg2": 789,
      "operation": "multiply"
}
    Result: {
      "arg1": 456,
      "arg2": 789,
      "operation": "multiply",
      "result": 359784,
      "success": true
}

[2] text_transformer
    Arguments: {
      "text": "python",
      "transform": "upper"
}
    Result: {
      "original": "python",
      "transformed": "PYTHON",
      "transform_type": "upper",
      "success": true
}



METHOD 2: Full ChatCompletion object

Model: mistralai/magistral-small-2509
Finish reason: stop
Final message: The 

**💡 Which method to use:**

**`client.print_tool_calls()`** - Best for:
- ✅ Quick debugging
- ✅ Learning and demonstrations
- ✅ Clean, readable output
- ✅ Shows both arguments AND results

**`return_full_response=True`** - Best for:
- ✅ Programmatic access to tool_calls
- ✅ Building automated workflows
- ✅ Full metadata (tokens, timing, etc.)
- ✅ Inspecting raw ChatCompletion structure

**Pro tip:** Use `print_tool_calls(detailed=True)` for full JSON inspection!

## 6️⃣ Combining Multiple Tools

The LLM can use multiple tools in a single conversation to solve complex tasks.

In [19]:
# Complex request requiring multiple tools
response = client.chat(
    "I have a text: 'The Quick BROWN fox'. "
    "Make it all lowercase, count the characters (no spaces), "
    "and if the character count is even, multiply it by 5, otherwise multiply by 3."
)

print("Result:", response)
client.print_tool_calls()

Result: Here are the results:

- Original text: 'The Quick BROWN fox'
- Lowercase text: 'the quick brown fox'
- Character count (no spaces): 19
- Since the character count is odd, it has been multiplied by 3, resulting in 57.

🔧 Tool Execution Summary (3 calls):
  [1] text_transformer(text=The Quick BROWN fox, transform=lower) → original=The Quick BROWN fox
  [2] char_counter(text=the quick brown fox) → text=the quick brown fox
  [3] math_calculator(arg1=15, arg2=3, operation=multiply) → result=45



**🧠 The LLM orchestrated:**
1. `text_transformer` to lowercase
2. `char_counter` to count (without spaces)
3. `math_calculator` to multiply based on even/odd

This shows the power of tools - the LLM becomes a coordinator!

## 7️⃣ Tools with Conversation History

Tools work seamlessly with conversation history.

In [21]:
# Start a conversation with tools
history = []

# Turn 1: Ask about a calculation
# NOTE: Some models may skip tools for simple math they can do mentally
response1, history = client.chat_with_history(
    "What is 25 * 16?",
    history,
    use_tools=True,
    tool_choice="required"
)
print("Turn 1:")
print(f"You: What is 25 * 16?")
print(f"LLM: {response1}")
print()
client.print_tool_calls()  # Show if tools were used

# Turn 2: Reference previous result  
response2, history = client.chat_with_history(
    "Now add 100 to that result.",
    history,
    use_tools=True,
    tool_choice="required"
)
print("\nTurn 2:")
print(f"You: Now add 100 to that result.")
print(f"LLM: {response2}")
print()
client.print_tool_calls()

# Turn 3: More complex operation
response3, history = client.chat_with_history(
    "What is 12847 multiplied by 9283? Be precise.",  # Changed to force tool use
    history,
    use_tools=True,
    tool_choice="required"
)
print("\nTurn 3:")
print(f"You: What is 12847 multiplied by 9283? Be precise.")
print(f"LLM: {response3}")
print()
client.print_tool_calls()

Turn 1:
You: What is 25 * 16?
LLM: The product of 25 and 16 is 400.


🔧 Tool Execution Summary (1 call):
  [1] math_calculator(arg1=25, arg2=16, operation=multiply) → result=400


Turn 2:
You: Now add 100 to that result.
LLM: The sum of 400 and 100 is 500.


🔧 Tool Execution Summary (1 call):
  [1] math_calculator(arg1=400, arg2=100, operation=add) → result=500


Turn 3:
You: What is 12847 multiplied by 9283? Be precise.
LLM: The precise product of 12847 multiplied by 9283 is 119,258,701.


🔧 Tool Execution Summary (1 call):
  [1] math_calculator(arg1=12847, arg2=9283, operation=multiply) → result=119258701



**💡 Important Model Behavior:**

Notice that NO tools were used in any of the turns above! This is because:
- **Magistral is a reasoning model** - It thinks through problems in `[THINK]` blocks
- During thinking, it solves problems mentally before deciding to use tools
- For simple math (25*16, 400+100), it concludes "I can do this" → no tool call

**The Solution: `tool_choice` Parameter**

Use `tool_choice="required"` to FORCE tool usage, bypassing internal reasoning:

```python
# Force tool usage with tool_choice="required"
response = client.chat(
    "What is 25 * 16?",
    use_tools=True,
    tool_choice="required"  # Forces the model to use a tool
)
```

Let's see this in action below! 👇

## 7️⃣b. Forcing Tool Usage with `tool_choice`

The `tool_choice` parameter controls whether and when the model uses tools:

In [20]:
print("=" * 70)
print("DEMO: tool_choice Parameter")
print("=" * 70)

# Test 1: tool_choice="auto" (default - model decides)
print("\n1️⃣ tool_choice='auto' (default):")
print("-" * 70)
response = client.chat(
    "What is 25 * 16?",
    use_tools=True,
    tool_choice="auto"  # Model decides
)
print(f"Response: {response}")
client.print_tool_calls()

# Test 2: tool_choice="required" (force tool use)
print("\n2️⃣ tool_choice='required' (force tool use):")
print("-" * 70)
response = client.chat(
    "What is 25 * 16?",
    use_tools=True,
    tool_choice="required"  # FORCE tool usage
)
print(f"Response: {response}")
client.print_tool_calls()

# Test 3: tool_choice="none" (prevent tool use)
print("\n3️⃣ tool_choice='none' (prevent tool use):")
print("-" * 70)
response = client.chat(
    "What is 25 * 16?",
    use_tools=True,
    tool_choice="none"  # Prevent tools
)
print(f"Response: {response}")
client.print_tool_calls()

print("\n" + "=" * 70)
print("✅ With tool_choice='required', the model MUST use a tool!")
print("=" * 70)

DEMO: tool_choice Parameter

1️⃣ tool_choice='auto' (default):
----------------------------------------------------------------------
Response: The result of multiplying 25 by 16 is 400.

🔧 Tool Execution Summary (1 call):
  [1] math_calculator(arg1=25, arg2=16, operation=multiply) → result=400


2️⃣ tool_choice='required' (force tool use):
----------------------------------------------------------------------
Response: The result of 25 multiplied by 16 is 400.

🔧 Tool Execution Summary (1 call):
  [1] math_calculator(arg1=25, arg2=16, operation=multiply) → result=400


3️⃣ tool_choice='none' (prevent tool use):
----------------------------------------------------------------------
Response: Let me calculate that for you.

25 * 16 = 400
ℹ️  No tools were called in the last request

✅ With tool_choice='required', the model MUST use a tool!


**📚 Understanding `tool_choice` Options:**

| Value | Behavior | Use When |
|-------|----------|----------|
| `"auto"` | Model decides if tools are needed | Default - balanced approach |
| `"required"` | Forces model to use at least one tool | Need guaranteed tool execution |
| `"none"` | Prevents all tool usage | Want pure LLM reasoning |

**⚠️ Trade-off with Reasoning Models:**
- `tool_choice="auto"`: Model thinks first, may skip tools for simple tasks
- `tool_choice="required"`: Bypasses thinking, goes straight to tool usage
- For **Magistral/reasoning models**: Use `"required"` when tools are mandatory

**When to use each:**
- **"auto"**: General use, let the smart model decide
- **"required"**: Calculator apps, API wrappers, guaranteed tool execution  
- **"none"**: Creative writing, brainstorming, pure reasoning tasks

**Advanced:** You can also force a specific tool:
```python
response = client.chat(
    "Calculate something",
    tool_choice={"type": "function", "function": {"name": "math_calculator"}}
)
```

## 🏋️ Exercise: Multi-Tool Text Analyzer

**Challenge:** Create a conversation that:
1. Takes a sample text: "The LOCAL LLM SDK makes AI Development EASIER!"
2. Converts it to lowercase
3. Counts characters with and without spaces
4. Counts words
5. Calculates the average character-per-word ratio

Requirements:
- Use at least 3 different built-in tools
- Show each step of the analysis
- Display the final statistics

Try it yourself first!

In [ ]:
# Your code here:



<details>
<summary>Click to see solution</summary>

```python
# Solution: Multi-tool text analyzer

sample_text = "The LOCAL LLM SDK makes AI Development EASIER!"

print("📝 Text Analysis Tool Demo\n")
print(f"Original text: {sample_text}")
print("="*70 + "\n")

# Analysis request
response = client.chat(
    f"Analyze this text: '{sample_text}'. "
    f"First, convert it to lowercase. "
    f"Then tell me: "
    f"(1) how many characters including spaces, "
    f"(2) how many characters excluding spaces, "
    f"(3) how many words, and "
    f"(4) calculate the average characters per word (chars without spaces / word count)."
)

print("🔍 Analysis Results:")
print(response)

# Alternative: Step by step with conversation history
print("\n" + "="*70)
print("\n📊 Step-by-Step Analysis:\n")

history = []

# Step 1: Lowercase
response1, history = client.chat_with_history(
    f"Convert to lowercase: '{sample_text}'",
    history
)
print(f"Step 1 - Lowercase: {response1}")

# Step 2: Character counts
response2, history = client.chat_with_history(
    "Count characters in that lowercased text, both with and without spaces.",
    history
)
print(f"\nStep 2 - Char counts: {response2}")

# Step 3: Word count
response3, history = client.chat_with_history(
    "How many words are in it?",
    history
)
print(f"\nStep 3 - Word count: {response3}")

# Step 4: Average
response4, history = client.chat_with_history(
    "Calculate the average characters per word (using chars without spaces).",
    history
)
print(f"\nStep 4 - Average: {response4}")
```
</details>

In [ ]:
# Solution cell (run this to see the answer)
sample_text = "The LOCAL LLM SDK makes AI Development EASIER!"

print("📝 Text Analysis Tool Demo\n")
print(f"Original text: {sample_text}")
print("="*70 + "\n")

# One-shot analysis
response = client.chat(
    f"Analyze this text: '{sample_text}'. "
    f"First, convert it to lowercase. "
    f"Then tell me: "
    f"(1) how many characters including spaces, "
    f"(2) how many characters excluding spaces, "
    f"(3) how many words, and "
    f"(4) calculate the average characters per word (chars without spaces / word count)."
)

print("🔍 Analysis Results:")
print(response)

# Step-by-step version
print("\n" + "="*70)
print("\n📊 Step-by-Step Analysis:\n")

history = []

response1, history = client.chat_with_history(
    f"Convert to lowercase: '{sample_text}'",
    history
)
print(f"Step 1 - Lowercase: {response1}")

response2, history = client.chat_with_history(
    "Count characters in that lowercased text, both with and without spaces.",
    history
)
print(f"\nStep 2 - Char counts: {response2}")

response3, history = client.chat_with_history(
    "How many words are in it?",
    history
)
print(f"\nStep 3 - Word count: {response3}")

response4, history = client.chat_with_history(
    "Calculate the average characters per word (using chars without spaces).",
    history
)
print(f"\nStep 4 - Average: {response4}")

## ⚠️ Common Pitfalls

### 1. Forgetting to Register Tools
```python
# ❌ Bad: Tools not registered
client = LocalLLMClient(base_url="...", model="...")
response = client.chat("Calculate 5 * 5")
# LLM tries to do math itself (often wrong)

# ✅ Good: Register tools first
client = LocalLLMClient(base_url="...", model="...")
client.register_tools_from(None)  # Load built-in tools
response = client.chat("Calculate 5 * 5")
```

### 2. Not Enabling Tools in Chat Call
```python
# ❌ Bad: Tools registered but not used
client.register_tools_from(None)
response = client.chat("Calculate 5 * 5")  # Tools available but not used

# ✅ Good: Enable tools in chat (default is use_tools=True)
response = client.chat("Calculate 5 * 5", use_tools=True)
# Or just: client.chat("Calculate 5 * 5") since use_tools=True is default
```

### 3. Model Doesn't Support Function Calling
```python
# ⚠️ Some models don't support function calling well
# Check your model's capabilities:
# - Qwen, Hermes, Functionary, Mistral: Good function calling
# - Older or smaller models: May not support it

# Test with a simple tool call to verify
response = client.chat("Calculate 123 * 456", use_tools=True)
# If answer is approximated or wrong, model may not support tools
```

### 4. Expecting Tools for Simple Tasks
```python
# ⚠️ For very simple math, LLM might not use tools
response = client.chat("What is 2 + 2?")
# LLM knows this and may answer directly: "4"

# Tools are used for:
# - Complex calculations: "What is 12847 * 9283?"
# - Precise operations: "Calculate exactly: 15.7 / 3.2"
# - Multi-step tasks: "Calculate (5+3) * (10-2) / 4"
```

## 🎓 What You Learned

✅ **Tool Concept**: Functions that extend LLM capabilities beyond text generation

✅ **Built-in Tools**: `math_calculator`, `text_transformer`, `char_counter`, `execute_python`, `filesystem_operation`, `get_weather`

✅ **Automatic Execution**: SDK handles tool calls transparently when `use_tools=True`

✅ **Tool Registration**: Use `client.register_tools_from(None)` to load built-in tools

✅ **Tool Inspection**: Use `return_full_response=True` to see tool_calls details

✅ **Multi-Tool Tasks**: LLM can orchestrate multiple tools for complex operations

✅ **Tools + History**: Combine tools with conversation context for powerful workflows

## 🚀 Next Steps

You've mastered built-in tools! Now let's create your own custom tools.

➡️ Continue to [05-custom-tools.ipynb](./05-custom-tools.ipynb) to learn how to:
- Create custom tools with the `@tool` decorator
- Define parameters with type hints
- Handle errors in tool functions
- Register and use your own tools
- Build a complete unit converter tool
- Follow best practices for tool design